# Fine-Tuning GPT

https://www.modeldifferently.com/en/2021/12/generaci%C3%B3n-de-fake-news-con-gpt-2/#data-loading-and-processing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch, os, re, pandas as pd, json
from sklearn.model_selection import train_test_split
from transformers import (
    DataCollatorForLanguageModeling,
    DataCollatorWithPadding,
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM
)
from datasets import Dataset, list_metrics, load_metric


In [3]:
#MODEL_NAME = "gpt2"
#model_cls = GPT2LMHeadModel
#tokenizer_cls = GPT2Tokenizer

MODEL_NAME = "microsoft/DialoGPT-small"
model_cls = AutoModelForCausalLM
tokenizer_cls = AutoTokenizer

In [4]:
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu"  

device = torch.device(dev) 

print(f"""
    cuda: {torch.cuda.is_available()}
    current_device: {torch.cuda.current_device()}
    device_count: {torch.cuda.device_count()}
""")


    cuda: True
    current_device: 0
    device_count: 1



In [5]:
config = AutoConfig.from_pretrained(MODEL_NAME)

## Load Data, Tokenizer, and Model

### Data Loading from CSV

In [6]:
filepath = "../data/processed.csv"
df = pd.read_csv(filepath, encoding="utf-8", usecols=["character", "content"]).rename(columns={"content": "text"})

pd.set_option("display.max_colwidth", None)
df.tail(10)

,character,text
32080,Charlotte,"What're the odds, right? But yes, we, uh...know each other. And he's also here in Boston."
32081,Toby,Right. Geeze. I...did not expect to ever hear from him again.
32082,Charlotte,...Surprise?
32083,Charlotte,"So, uh, would you want to talk to him? Because this is all stuff he really should get the chance to ask you about directly."
32084,Toby,Um...sure.
32085,Charlotte,"Ok, good. Um, we should probably go. But, uh, I'll see you tomorrow?"
32086,Charlotte,[_as she talks she takes the book off the table and carefully tucks it away in her bag_]
32087,Toby,"Um, yeah."
32088,Toby,Tomorrow.
32089,Charlotte,[_she will leave with the others_]


### Model

In [7]:
base_model = model_cls.from_pretrained(MODEL_NAME, config=config)

In [8]:
base_model.num_parameters

<bound method ModuleUtilsMixin.num_parameters of GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0

### Tokenizer

In [15]:
base_tokenizer = tokenizer_cls.from_pretrained(MODEL_NAME)

In [10]:
print(f"vocab_size: {base_tokenizer.vocab_size}")

vocab_size: 50257


In [11]:
vocabulary = base_tokenizer.get_vocab()
vocabulary["Hi"]

17250

In [12]:
base_tokenizer.eos_token

'<|endoftext|>'

## Conversational Model

### Data Preparation

In [13]:
import argparse
import tarfile
import os.path

from typing import Dict, Union

import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoModelWithLMHead, AutoTokenizer
from datasets import Dataset


def load_csv(filename):
    return pd.read_csv(filename)


def prepare_data(
    data_or_filename: Union[str, pd.DataFrame],
    filter_by: str = None,
    filter_value: str = None,
    content_key: str = "text",
    n: int = 7,
    test_size: float = 0.1,
    flatten: bool = True,
):
    data = load_csv(data_or_filename) if isinstance(data_or_filename, str) else data_or_filename

    contexted_data = prepare_context(
        data,
        filter_by=filter_by,
        filter_value=filter_value,
        content_key=content_key,
        n=n,
    )

    trn_df, val_df = train_test_split(contexted_data, test_size=test_size)
    
    if flatten:
        train_dataset = prepare_dataset(trn_df)
        val_dataset = prepare_dataset(val_df)
        return train_dataset, val_dataset

    return trn_df, val_df


def prepare_context(
    data: pd.DataFrame,
    filter_by: str = None,
    filter_value: str = None,
    content_key: str = "text",
    n: int = 7,
):
    if filter_by:
        indexes = data.loc[data[filter_by] == filter_value].index
    else:
        indexes = range(n, len(data[content_key]))

    contexted = []

    for i in indexes:
        row = []
        prev = i - 1 - n
        for j in range(i, prev, -1):
            row.append(data.iloc[j][content_key])
        contexted.append(row)

    columns = ["response", "context"]
    columns = columns + ["context/" + str(i) for i in range(n - 1)]

    df = pd.DataFrame.from_records(contexted, columns=columns)
    return df


def prepare_dataset(df):
    columns = [col for col in df] 
    dataset = Dataset.from_pandas(concat_text(df))
    dataset = dataset.remove_columns(columns + ['__index_level_0__'])
    return dataset


def concat_text(df):
    df["text"] = df.apply(concat_text_in_row, axis=1)
    return df


def concat_text_in_row(row):
    concat_text = f"{base_tokenizer.eos_token}".join(row.values)
    # Add to end
    concat_text += base_tokenizer.eos_token
    return concat_text


def construct_conv(example, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x, padding="max_length", max_length=250) + [tokenizer.eos_token_id] for x in example]))
    print(f"Conv Length: {len(conv)}") 
    print(set(list(map(len, conv))))
    conv = flatten(conv)
    return conv

In [14]:
#base_tokenizer.pad_token = "<|PAD|>"

In [18]:
base_tokenizer._pad_token

In [19]:
train_df, val_df = prepare_data(df, filter_by="character", filter_value="bitjockey", flatten=False)

In [20]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [21]:
def preprocess(tokenizer):
    def _tokenize(example):
        flatten = lambda l: [item for sublist in l for item in sublist]
        conv = list(reversed([tokenizer.encode(x, padding="max_length", max_length=250) + [tokenizer.eos_token_id] for x in example if "__index" not in x]))
        conv = flatten(conv)
        return conv
        
    def _preprocess(examples):
        #flatten = lambda l: [item for sublist in l for item in sublist]
        #conv = list([tokenizer.encode(x, padding="max_length", max_length=250) + [tokenizer.eos_token_id] for x in examples])
        #conv = flatten(conv)
        #return {"input_ids": conv}
        return {"input_ids": list(map(_tokenize, examples))}
    
    return _preprocess


def tokenize(example):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x, padding="max_length", max_length=250) + [tokenizer.eos_token_id] for x in example]))
    print(f"Conv Length: {len(conv)}") 
    print(set(list(map(len, conv))))
    conv = flatten(conv)
    return conv


def debug_preprocess(tokenizer, max_length=250): 
    def _tokenize(examples):
        input_ids = [tokenizer.encode(v, padding="max_length", max_length=250) for k, v in examples.items() if isinstance(v, str)]
        examples["input_ids"] = input_ids
        return examples
    return _tokenize

In [22]:
train_dataset

Dataset({
    features: ['response', 'context', 'context/0', 'context/1', 'context/2', 'context/3', 'context/4', 'context/5', '__index_level_0__'],
    num_rows: 4967
})

In [23]:
val_dataset

Dataset({
    features: ['response', 'context', 'context/0', 'context/1', 'context/2', 'context/3', 'context/4', 'context/5', '__index_level_0__'],
    num_rows: 552
})

In [62]:
from typing import List

from torch.nn.utils.rnn import pad_sequence


def preprocess_conv(tokenizer):
    def _construct(examples):
        flatten = lambda l: [item for sublist in l for item in sublist]
        conv = list(reversed([tokenizer.encode(v) + [tokenizer.eos_token_id] for k,v in examples.items() if isinstance(v, str)]))
        conv = flatten(conv)
        examples["input_ids"] = conv
        return examples 
    return _construct


def collate(examples: List[torch.Tensor], tokenizer):
    if tokenizer._pad_token is None:
        return pad_sequence(examples, batch_first=True)
    return pad_sequence(
        examples, batch_first=True, padding_value=tokenizer.pad_token_id
    )


In [55]:
base_tokenizer

PreTrainedTokenizerFast(name_or_path='microsoft/DialoGPT-small', vocab_size=50257, model_max_len=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True)})

In [60]:
tokenized_val_dataset = val_dataset.map(preprocess_conv(tokenizer=base_tokenizer), remove_columns=list(val_dataset.features.keys()))

  0%|          | 0/552 [00:00<?, ?ex/s]

In [57]:
len(tokenized_val_dataset["input_ids"][1])

100

In [58]:
base_tokenizer.decode(tokenized_val_dataset["input_ids"][1])

"Right. She..blames herself. [_makes a face_]...are you..suggesting I...[_waves hand around_] just..._tell_ her?<|endoftext|>That you need some help with this and I suggested this?<|endoftext|>...yes.<|endoftext|>Does that make you too uncomfortable?<|endoftext|>A bit. [_lopsided grin_]<|endoftext|>It's just a thought.<|endoftext|>Worth a try.<|endoftext|>[_considers_] I think I will try that.<|endoftext|>"

In [65]:
tokenized_val_dataset.set_format(type="torch", columns=["input_ids"])

In [66]:
collate(tokenized_val_dataset["input_ids"], base_tokenizer)

tensor([[ 2396,   986, 33331,  ...,     0,     0,     0],
        [11028,    13,  1375,  ...,     0,     0,     0],
        [29795, 39624,   306,  ...,     0,     0,     0],
        ...,
        [15496,    13, 50256,  ...,     0,     0,     0],
        [ 5661,  3404,  2476,  ...,     0,     0,     0],
        [ 1106, 11028,    11,  ...,     0,     0,     0]])

In [67]:
tokenized_val_dataset

Dataset({
    features: ['input_ids'],
    num_rows: 552
})

In [25]:
tokenized_train_dataset = train_dataset.map(debug_preprocess(base_tokenizer), remove_columns=list(train_dataset.features.keys()))

  0%|          | 0/4967 [00:00<?, ?ex/s]

In [26]:
len(tokenized_train_dataset["input_ids"][0])

8

In [27]:
tokenized_train_dataset.set_format(type="torch", columns=["input_ids"])
tokenized_val_dataset.set_format(type="torch", columns=["input_ids"])

In [28]:
len(tokenized_train_dataset["input_ids"][0])

8

### Metrics

In [29]:
metric = load_metric("perplexity")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


### Training

In [183]:
base_model.resize_token_embeddings(len(base_tokenizer))

Embedding(50257, 768)

In [184]:
FINETUNED_MODEL = 'myDialoGPT2-small'

data_collator = DataCollatorForLanguageModeling(
    tokenizer=base_tokenizer,
    mlm=False,
)

trainer = None
training_args = TrainingArguments(
    output_dir=FINETUNED_MODEL,          # output directory
    evaluation_strategy="epoch",
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=1,  # batch size per device during training
    per_device_eval_batch_size=1,   # batch size for evaluation
    weight_decay=0.01,               # strength of weight decay
    logging_dir=FINETUNED_MODEL,            # directory for storing logs
    prediction_loss_only=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [185]:
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [186]:
trainer.get_train_dataloader()

In [187]:
trainer.train()

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4967
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 14901


Epoch,Training Loss,Validation Loss


Saving model checkpoint to myDialoGPT2-small/checkpoint-500
Configuration saved in myDialoGPT2-small/checkpoint-500/config.json


Attempted to log scalar metric loss:
4.7985
Attempted to log scalar metric learning_rate:
4.832226025098987e-05
Attempted to log scalar metric epoch:
0.1


Model weights saved in myDialoGPT2-small/checkpoint-500/pytorch_model.bin


KeyboardInterrupt: 

In [61]:
trainer.save_model(FINETUNED_MODEL)
base_tokenizer.save_pretrained(FINETUNED_MODEL)
config.save_pretrained(FINETUNED_MODEL)

Saving model checkpoint to my-DialoGPT
Configuration saved in my-DialoGPT/config.json
Model weights saved in my-DialoGPT/pytorch_model.bin
tokenizer config file saved in my-DialoGPT/tokenizer_config.json
Special tokens file saved in my-DialoGPT/special_tokens_map.json
Configuration saved in my-DialoGPT/config.json


In [62]:
finetuned_model = AutoModelForCausalLM.from_pretrained(FINETUNED_MODEL)
tokenizer = AutoTokenizer.from_pretrained(FINETUNED_MODEL)

loading configuration file my-DialoGPT/config.json
Model config GPT2Config {
  "_name_or_path": "my-DialoGPT",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "conversational": {
      "max_length": 1000
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.19.0",
  "use_cache": true,
  "vocab_size": 50257
}

loading weights file my-DialoGPT/p

In [23]:
def generate_responses(model, tokenizer, text, chat_history_ids=None, step=0):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(text + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=512,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=100,
        top_p=0.7
    )
    
    response = tokenizer.decode(
        chat_history_ids[:, bot_input_ids.shape[-1]:][0],
        skip_special_tokens=True,
    )

    return response, chat_history_ids, step + 1

def chat(model, tokenizer):
    step = 0
    chat_history_ids = []
    
    while True: 
        text = input(">> ")
        if text in ["/q", "/quit", "/e", "/exit"]: break
        print(f"User: {text}")
        response, chat_history_ids, step = generate_responses(
            model=model,
            tokenizer=tokenizer,
            text=text,
            chat_history_ids=chat_history_ids,
            step=step
        )
        print(f"Bot: {response}")
        

In [24]:
chat(finetuned_model, tokenizer)

>>  Hello


User: Hello
Bot: Hey. You're on the floor. [*to hopper*] How are you? [*to Charlotte*] Hey. [_to Amarok, Kalahan, and Hopper_] What happened? [*charlotte doesn't say anything, just looks around at everyone*] Hi. How are..you? [*will follow*] Oh hey Hopper. How about you? Have a nice day? [*gives hopper a quick hug*] I'll be right back, if you need me. [*looks around*] [_will head to the other room_] It's a little late for this, but if you want to come with, I'm in the common room, the people who are in the building are also here. [ _to Hopper, Kalahashi_]...Hey, are you coming? [*shakes head*]  Oh, hi.  Oh hey, Charlotte.  Sorry, I haven't seen you in awhile.  [_she takes the commlink off the couch and sits on the couch, reading the book, just reading it._]  I can meet you there if you're ready.  How about yourself, uhm, at the library.  Yeah, I'll go.  I'll let you know when I'm ready. [*sighs_]  Hey, did you want anything, you want me to come over?  I have something to sit down, or d

>>  /quit


In [8]:
text = "[_as she talks she takes the book off the table and carefully tucks it away in her bag_]"

In [15]:
tokenizer.pad_token = "<|pad|>"
tokenizer(text, padding="max_length", max_length=512)

{'input_ids': [29795, 292, 673, 6130, 673, 2753, 262, 1492, 572, 262, 3084, 290, 7773, 256, 6238, 340, 1497, 287, 607, 6131, 62, 60, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,

In [14]:
finetuned_model.max_length

AttributeError: 'GPT2LMHeadModel' object has no attribute 'max_length'